In [6]:
array_ts = []
array_ts.append({1:123})
print(array_ts)
array_ts.append({2:456})
print(array_ts)

[{1: 123}]
[{1: 123}, {2: 456}]


In [22]:
d1 = float('-inf')
d2 = float('-inf')
if d1 == d2 :
    print("yes")

yes


In [34]:
for i in range(1,10):
    print(i)
for i in range(0,8,2):
    print(i)

1
2
3
4
5
6
7
8
9
0
2
4
6


In [3]:
x = [1]
y = [1,2,3]
y.extend(x)
y

[1, 2, 3, 1]

In [27]:
import numpy as np
xx = [0.23818429, 0.23818429, 0.23818429, 0.23818429, 0.23818429]
print(round(np.std(xx,ddof=0),2))
#

0.0
